In [ ]:
from openai import OpenAI
import json
from tqdm import tqdm

In [ ]:
def translator(query):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "请作为一个翻译软件，将输入的文字翻译为中文。"
            },
            {
                "role": "user",
                "content": query
            }
        ],
        model="deepseek-v3",
        max_tokens=1024,
        temperature=0.1,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1
    )
    return chat_completion.choices[0].message.content

In [ ]:
dataset = json.load(open("../../output/o3_conv.json", 'r'))
for data in tqdm(dataset):
    for conv in data['conversation']:
        content = conv['content']
        pattern = r"> Reasoning(.*?)Reasoned for a couple of seconds"
        matches = re.findall(pattern, content, re.DOTALL)
        if matches:
            think_pattern = matches[0]
            think_pattern = ">" + translator(think_pattern)
            conv['content'] = re.sub(pattern, think_pattern, content, flags=re.DOTALL)

In [ ]:
json.dump(dataset, open("../../output/o3_conv_trans.json", 'w'), ensure_ascii=False)